In [3]:
import pandas as pd
import math
import os.path
import time
from bitmex import bitmex
# from binance.client import Client
from datetime import timedelta, datetime, timezone
from dateutil import parser, relativedelta
from tqdm import tqdm

### API
bitmex_api_key = 'vcrjOJKb6SRv0ALqJiQ6nBTV'    #Enter your own API-key here
bitmex_api_secret = 'Zm2wNNjDXIlSSlKztGrwXKxthcAPzX-Wy0ScYjB2QeQK3dpg' #Enter your own API-secret here
binance_api_key = '[REDACTED]'    #Enter your own API-key here
binance_api_secret = '[REDACTED]' #Enter your own API-secret here

### CONSTANTS
binsizes = {"1m": 1, "5m": 5, "1h": 60, "1d": 1440, '1q': 131040}
batch_size = 750
bitmex_client = bitmex(test=False, api_key=bitmex_api_key, api_secret=bitmex_api_secret)
# binance_client = Client(api_key=binance_api_key, api_secret=binance_api_secret)


### FUNCTIONS
def minutes_of_new_data(symbol, kline_size, data, source):
    if len(data) > 0:  old = parser.parse(data["timestamp"].iloc[-1])
    # elif source == "binance": old = datetime.strptime('1 Jan 2017', '%d %b %Y')
    elif source == "bitmex": 
        # get a date one quarter back
        now = datetime.now(timezone.utc)
        old = now - relativedelta.relativedelta(months=12)
    # if source == "binance": new = pd.to_datetime(binance_client.get_klines(symbol=symbol, interval=kline_size)[-1][0], unit='ms')
    if source == "bitmex": new = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=True).result()[0][0]['timestamp']
    return old, new

# def get_all_binance(symbol, kline_size, save = False):
#     filename = '%s-%s-data.csv' % (symbol, kline_size)
#     if os.path.isfile(filename): data_df = pd.read_csv(filename)
#     else: data_df = pd.DataFrame()
#     oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "binance")
#     delta_min = (newest_point - oldest_point).total_seconds()/60
#     available_data = math.ceil(delta_min/binsizes[kline_size])
#     if oldest_point == datetime.strptime('1 Jan 2017', '%d %b %Y'): print('Downloading all available %s data for %s. Be patient..!' % (kline_size, symbol))
#     else: print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data.' % (delta_min, symbol, available_data, kline_size))
#     klines = binance_client.get_historical_klines(symbol, kline_size, oldest_point.strftime("%d %b %Y %H:%M:%S"), newest_point.strftime("%d %b %Y %H:%M:%S"))
#     data = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
#     data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
#     if len(data_df) > 0:
#         temp_df = pd.DataFrame(data)
#         data_df = data_df.append(temp_df)
#     else: data_df = data
#     data_df.set_index('timestamp', inplace=True)
#     if save: data_df.to_csv(filename)
#     print('All caught up..!')
#     return data_df

def get_all_bitmex(symbol, kline_size, save = False):
    filename = '../%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): data_df = pd.read_csv(filename)
    else: data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "bitmex")
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    rounds = math.ceil(available_data / batch_size)
    if rounds > 0:
        print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data in %d rounds.' % (delta_min, symbol, available_data, kline_size, rounds))
        for round_num in tqdm(range(rounds)):
            time.sleep(1)
            new_time = (oldest_point + timedelta(minutes = round_num * batch_size * binsizes[kline_size]))
            data = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=batch_size, startTime = new_time).result()[0]
            temp_df = pd.DataFrame(data)
            data_df = data_df.append(temp_df)
    data_df.set_index('timestamp', inplace=True)
    if save and rounds > 0: data_df.to_csv(filename)
    print('All caught up..!')
    return data_df


get_all_bitmex('XBTUSD', '5m', save=True)


  0%|          | 0/141 [00:00<?, ?it/s]

100%|██████████| 141/141 [03:09<00:00,  1.34s/it]


All caught up..!


,symbol,open,high,low,close,trades,volume,vwap,lastSize,turnover,homeNotional,foreignNotional
timestamp,,,,,,,,,,,,
2019-03-01 12:20:00+00:00,XBTUSD,3806.5,3807.0,3806.5,3806.5,171,491889,3806.7684,12500,12921608255,129.216083,491889.0
2019-03-01 12:25:00+00:00,XBTUSD,3806.5,3807.0,3806.5,3807.0,371,225640,3806.9134,6,5927175616,59.271756,225640.0
2019-03-01 12:30:00+00:00,XBTUSD,3807.0,3808.5,3806.5,3808.0,775,2951827,3807.6381,200,77526291601,775.262916,2951827.0
2019-03-01 12:35:00+00:00,XBTUSD,3808.0,3808.5,3806.5,3807.0,954,2053020,3807.6381,3,53918864080,539.188641,2053020.0
2019-03-01 12:40:00+00:00,XBTUSD,3807.0,3807.0,3806.5,3807.0,306,406969,3806.7684,20,10690915747,106.909157,406969.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-01 11:55:00+00:00,XBTUSD,8642.5,8656.0,8642.0,8656.0,3720,17235734,8647.5268,4,199324262828,1993.242628,17235734.0
2020-03-01 12:00:00+00:00,XBTUSD,8656.0,8658.0,8646.0,8652.5,3376,20881431,8652.7646,103,241338709982,2413.387100,20881431.0
2020-03-01 12:05:00+00:00,XBTUSD,8652.5,8769.0,8652.5,8742.5,20359,93355000,8717.6358,5000,1070883973937,10708.839739,93355000.0


In [6]:
df = pd.read_csv('/home/kn/Documents/thesis/XBTUSD-5m-data.csv')

df15 = df.iloc[::3, :]
df30 = df.iloc[::6, :]

df15.to_csv('/home/kn/Documents/thesis/XBTUSD-15m-data.csv')
df30.to_csv('/home/kn/Documents/thesis/XBTUSD-30m-data.csv')